In [2]:
import asyncio
import logging
import os

from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from semantic_kernel.contents import StreamingFileReferenceContent

logging.basicConfig(level=logging.ERROR)

In [ ]:
import asyncio
import logging
import os

from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from semantic_kernel.contents import StreamingFileReferenceContent

logging.basicConfig(level=logging.ERROR)

# Use the current working directory (notebook directory) instead of __file__
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
csv_file_path_1 = os.path.join(parent_dir, "data_processing", "csv_tables", "merged_analysis_of_financial_experience.csv")
csv_file_path_2 = os.path.join(parent_dir, "data_processing", "csv_tables", "merged_schedules_of_changes_in_net_pension_liability.csv")

async def download_file_content(agent: AzureAssistantAgent, file_id: str):
    try:
        # Fetch the content of the file using the provided method
        response_content = await agent.client.files.content(file_id)

        # Use the current working directory for file saving
        file_path = os.path.join(
            os.getcwd(),  # current working directory
            f"{file_id}.png",  # modify as needed for file extension or naming
        )

        # Save content to a file
        with open(file_path, "wb") as file:
            file.write(response_content.content)

        print(f"File saved to: {file_path}")
    except Exception as e:
        print(f"An error occurred while downloading file {file_id}: {str(e)}")


async def download_response_image(agent: AzureAssistantAgent, file_ids: list[str]):
    if file_ids:
        for file_id in file_ids:
            await download_file_content(agent, file_id)


async def main():
    # Create the client using Azure OpenAI resources and configuration
    client, model = AzureAssistantAgent.setup_resources()

    # Upload the files to the client
    file_ids: list[str] = []
    for path in [csv_file_path_1, csv_file_path_2]:
        with open(path, "rb") as file:
            file = await client.files.create(file=file, purpose="assistants")
            file_ids.append(file.id)

    # Get the code interpreter tool and resources
    code_interpreter_tools, code_interpreter_tool_resources = AzureAssistantAgent.configure_code_interpreter_tool(
        file_ids=file_ids
    )

    # Create the assistant definition
    definition = await client.beta.assistants.create(
        model=model,
        instructions="""
            Analyze the available data to provide an answer to the user's question.
            Always format response using markdown.
            Always include a numerical index that starts at 1 for any lists or tables.
            Always sort lists in ascending order.
            """,
        name="SampleAssistantAgent",
        tools=code_interpreter_tools,
        tool_resources=code_interpreter_tool_resources,
    )

    # Create the agent using the client and the assistant definition
    agent = AzureAssistantAgent(
        client=client,
        definition=definition,
    )

    thread: AssistantAgentThread = None

    try:
        is_complete: bool = False
        file_ids: list[str] = []
        while not is_complete:
            user_input = input("User:> ")
            if not user_input:
                continue

            if user_input.lower() == "exit":
                is_complete = True
                break

            is_code = False
            last_role = None
            async for response in agent.invoke_stream(messages=user_input, thread=thread):
                current_is_code = response.metadata.get("code", False)

                if current_is_code:
                    if not is_code:
                        print("\n\n```python")
                        is_code = True
                    print(response.content, end="", flush=True)
                else:
                    if is_code:
                        print("\n```")
                        is_code = False
                        last_role = None
                    if hasattr(response, "role") and response.role is not None and last_role != response.role:
                        print(f"\n# {response.role}: ", end="", flush=True)
                        last_role = response.role
                    print(response.content, end="", flush=True)
                file_ids.extend([
                    item.file_id for item in response.items if isinstance(item, StreamingFileReferenceContent)
                ])
                thread = response.thread
            if is_code:
                print("```\n")
            print()

            await download_response_image(agent, file_ids)
            file_ids.clear()

    finally:
        print("\nCleaning up resources...")
        # Delete uploaded files
        [await client.files.delete(file_id) for file_id in file_ids]
        if thread:
            await thread.delete()
        await client.beta.assistants.delete(agent.id)

# Run the main coroutine in the notebook
await main()



# assistant: To analyze the net pension liability change over the years, I need to inspect the contents and structure of the files you uploaded. I’ll start by reviewing the files to extract relevant data for the analysis.

```python
# Checking the contents and structure of the first uploaded file.
import pandas as pd

file_path_1 = '/mnt/data/assistant-FgWocR7ZH4caqxofsMcrU5'
file_path_2 = '/mnt/data/assistant-Nu5NzJhjsGnVrozbtbKJri'

data_1 = pd.ExcelFile(file_path_1)
data_1.sheet_names  # Checking the available sheets in the file
```

# assistant: It seems the file is not recognized as an Excel file. I'll attempt to inspect its content to determine the correct format and proceed accordingly.

```python
# Checking the raw file type and previewing its content to identify its format.
import mimetypes

file_type_1 = mimetypes.guess_type(file_path_1)
file_type_2 = mimetypes.guess_type(file_path_2)

file_type_1, file_type_2
```

# assistant: The file format is unclear from the guessed typ